In [4]:
# H20 is used for native handling of categorical data.
import h2o 
from h2o.estimators import H2ORandomForestEstimator as rfe
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\Alexander Saunders\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALEXAN~1\AppData\Local\Temp\tmpt7yje9_7
  JVM stdout: C:\Users\ALEXAN~1\AppData\Local\Temp\tmpt7yje9_7\h2o_Alexander_Saunders_started_from_python.out
  JVM stderr: C:\Users\ALEXAN~1\AppData\Local\Temp\tmpt7yje9_7\h2o_Alexander_Saunders_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_Alexander_Saunders_jd59sv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.531 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [5]:
data_path = "usda_plantsdb.csv"

# Read the mouse data and the study results
data = pd.read_csv(data_path,low_memory=False)
# Combine the data into a single dataset
df = pd.DataFrame(data)
column_list = list(df)
df_null_list = df.isnull().sum().sort_values().head(100)


In [104]:
df_core = pd.DataFrame(df, columns = [
 'Accepted_Symbol_x',
 'Hybrid_Genus_Indicator',
 'Hybrid_Species_Indicator',
 'Species',
 'Subspecies_Prefix',
 'Variety',
 'Genera_Binomial_Author',
 'Common_Name',
 'State_and_Province',
 'Category',
 'Family_Symbol',
 'Family_Common_Name',
 'xOrder',
 'SubClass',
 'Class',
 'Division',
 'SuperDivision',
 'SubKingdom',
 'Kingdom',
 'Duration',
 'Growth_Habit',
 'Native_Status',
 'Federal_Noxious_Status',
 'State_Noxious_Status',
 'Invasive',
 'Federal_T_E_Status',
 'State_T_E_Status',
 'Genus'])

In [129]:
df_core['Family_Common_Name'].unique()

array(['Mallow family', 'unknown', "Four o'clock family", 'Pine family',
       'Honeysuckle family', 'Sedge family', 'Pea family', 'Aster family',
       'Rose family', 'Spurge family', 'Calamus family', 'Grass family',
       'Buttercup family', 'Mint family', 'Cactus family',
       'Acanthus family', 'Potato family', 'Chinese Gooseberry family',
       'Amaranth family', 'Maidenhair Fern family', 'Maple family',
       'Barberry family', 'Pink family', 'Carrot family', 'Rue family',
       'Calycera family', 'Iris family', 'Melastome family',
       'Palm family', 'Curly-grass family', 'Kihi Fern family',
       'Kapok-tree family', 'Buckthorn family', 'Dogbane family',
       'Fumitory family', 'Moschatel family', 'Stonecrop family',
       'Horse-chestnut family', 'Sapodilla family', 'Bromeliad family',
       'Verbena family', 'Broom-rape family', 'Podocarpus family',
       'Ginger family', 'Figwort family', 'Lily family',
       'Century-plant family', 'Heath family', 'Araucar

In [105]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [106]:
df_core.fillna('unknown', inplace=True)

In [107]:
def in_state(string, state):
        if string.find(state) == -1:
            return 0
        else:
            return 1

In [108]:
for state in states:
    df_core[state] = df_core['State_and_Province'].apply(lambda x: in_state(x, state))

In [109]:
df_core.drop(columns=['State_and_Province'], inplace=True)

In [110]:
df_core['Invasive_In_NC'] = df_core['State_Noxious_Status'].apply(lambda x: in_state(x, 'NC'))

In [111]:
df_core['Invasive_In_NC'].sum()

110

In [112]:
df_core['Growth_Habit'] = df_core['Growth_Habit'].str.split(',',expand=True)[0]

In [113]:
df_core['Growth_Habit'].head()

0       Subshrub
1        unknown
2        unknown
3    Nonvascular
4        unknown
Name: Growth_Habit, dtype: object

In [114]:
df_core['Common_Name'] = df_core['Common_Name'].apply(lambda x: 0 if x == 'unknown' else 1)

In [115]:
df_core['Common_Name'].value_counts()

0    48971
1    43200
Name: Common_Name, dtype: int64

In [116]:
df_core['Invasive'].value_counts()

unknown                               90735
STATE                                   248
SWSS                                    131
HEAR                                    109
STATE, US                                85
                                      ...  
FLEPPC, STATE, SWSS, US                   1
Cal-IPC, N'EAST, NE&GP, SWSS, WSWS        1
KY, N'EAST, STATE, SWSS, WI               1
NE&GP, WI                                 1
N'EAST, SEEPPC, STATE, SWSS, WI           1
Name: Invasive, Length: 167, dtype: int64

In [117]:
df_core['Duration'].value_counts()

unknown                        59943
Perennial                      25296
Annual                          4923
Annual, Perennial                813
Biennial, Perennial              426
Annual, Biennial                 339
Annual, Biennial, Perennial      225
Biennial                         204
Annual, Perennial, Biennial        1
Perennial, Annual                  1
Name: Duration, dtype: int64

In [118]:
duration_types = ['Perennial', 'Annual', 'Biennial']
for dur in duration_types:
    df_core[dur] = df_core['Duration'].apply(lambda x: in_state(x, dur))

In [119]:
df_core.columns

Index(['Accepted_Symbol_x', 'Hybrid_Genus_Indicator',
       'Hybrid_Species_Indicator', 'Species', 'Subspecies_Prefix', 'Variety',
       'Genera_Binomial_Author', 'Common_Name', 'Category', 'Family_Symbol',
       'Family_Common_Name', 'xOrder', 'SubClass', 'Class', 'Division',
       'SuperDivision', 'SubKingdom', 'Kingdom', 'Duration', 'Growth_Habit',
       'Native_Status', 'Federal_Noxious_Status', 'State_Noxious_Status',
       'Invasive', 'Federal_T_E_Status', 'State_T_E_Status', 'Genus', 'AL',
       'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID',
       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
       'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
       'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
       'WI', 'WY', 'Invasive_In_NC', 'Perennial', 'Annual', 'Biennial'],
      dtype='object')

In [120]:
hf = h2o.H2OFrame(df_core)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [121]:
hf['Invasive'] = hf['Invasive'].asfactor()
predictors=['Accepted_Symbol_x', 'Hybrid_Genus_Indicator',
       'Hybrid_Species_Indicator', 'Variety',
       'Genera_Binomial_Author', 'Common_Name', 'Category', 'Family_Symbol',
       'Family_Common_Name', 'xOrder', 'SubClass', 'Class', 'Division',
       'SuperDivision', 'SubKingdom', 'Kingdom', 'Growth_Habit', 'Genus', 'AL',
       'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID',
       'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
       'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
       'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
       'WI', 'WY','Perennial', 'Annual', 'Biennial']
response='Invasive_In_NC'
train, valid = hf.split_frame(ratios=[0.7], seed = 42)

In [122]:
plants_drf = rfe(ntrees=100, max_depth =5, min_rows=10, balance_classes=True, binomial_double_trees=True)

In [123]:
plants_drf.train(x=predictors, 
                 y=response,
               training_frame=train,
               validation_frame=valid)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1636506170118_7


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,49013.0,5.0,5.0,5.0,9.0,22.0,13.92




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.0012456045218412755
RMSE: 0.035293122868928384
MAE: 0.002491984412212874
RMSLE: 0.02449142724371071
Mean Residual Deviance: 0.0012456045218412755

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 0.0010034974100052937
RMSE: 0.031678027242953335
MAE: 0.0022464017814842964
RMSLE: 0.022028377889307064
Mean Residual Deviance: 0.0010034974100052937

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-11-09 22:17:43,0.036 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2021-11-09 22:17:43,0.108 sec,1.0,0.035457,0.002464,0.001257,0.029809,0.002023,0.000889
2,,2021-11-09 22:17:43,0.161 sec,2.0,0.035974,0.002522,0.001294,0.030152,0.002111,0.000909
3,,2021-11-09 22:17:43,0.203 sec,3.0,0.036104,0.002597,0.001304,0.030652,0.002218,0.000940
4,,2021-11-09 22:17:44,0.246 sec,4.0,0.036684,0.002597,0.001346,0.030860,0.002174,0.000952
5,,2021-11-09 22:17:44,0.290 sec,5.0,0.035919,0.002552,0.001290,0.030985,0.002198,0.000960
6,,2021-11-09 22:17:44,0.338 sec,6.0,0.036016,0.002541,0.001297,0.031106,0.002193,0.000968
7,,2021-11-09 22:17:44,0.388 sec,7.0,0.035688,0.002525,0.001274,0.031188,0.002209,0.000973
8,,2021-11-09 22:17:44,0.436 sec,8.0,0.035675,0.002515,0.001273,0.031317,0.002215,0.000981
9,,2021-11-09 22:17:44,0.487 sec,9.0,0.035540,0.002511,0.001263,0.031399,0.002223,0.000986



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Family_Symbol,84.594818,1.000000,0.326842
1,Family_Common_Name,58.331989,0.689546,0.225373
2,Accepted_Symbol_x,29.757607,0.351766,0.114972
3,xOrder,18.844202,0.222758,0.072807
4,Genus,15.412136,0.182188,0.059547
5,Genera_Binomial_Author,4.880306,0.057690,0.018856
6,Growth_Habit,4.162879,0.049210,0.016084
7,HI,3.264380,0.038588,0.012612
8,SubClass,3.207190,0.037912,0.012391
9,MT,2.650129,0.031327,0.010239



See the whole table with table.as_data_frame()


In [124]:
perf = plants_drf.model_performance()

In [125]:
perf


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.0012456045218412755
RMSE: 0.035293122868928384
MAE: 0.002491984412212874
RMSLE: 0.02449142724371071
Mean Residual Deviance: 0.0012456045218412755
